In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

ball_by_ball_data = pd.read_csv("data/deliveries.csv")
matches_data = pd.read_csv("data/matches.csv")

In [ ]:
matches_data['match_id'] = matches_data['id']
matches_data.drop('id', axis = 1, inplace=True)

In [ ]:
data = pd.merge(ball_by_ball_data, matches_data, on='match_id', how='left')
data

In [ ]:
data.columns

In [ ]:
#ensuring that wides, noballs, and penalties don't count as a ball
for i in range(len(data)):
    if data.extras_type[i] in ['wides', 'noballs', 'penalty']:
        k = True
        j = i+1
        # Ensure j does not exceed DataFrame length
        while k and j < len(data):
            if data.ball[j] > data.ball[i]:
                data.loc[j, 'ball'] = data.ball[j] - 1
                j += 1
            else:
                k = False
        data.loc[i, 'ball'] = data.ball[i] - 1


In [ ]:
#get the cumulative innings total runs
innings_total_list = []
i = 0
x = True
while i < len(data):
    innings_total = data.total_runs[i]
    innings_total_list.append(innings_total)
    i = i+1
    while i < len(data) and data.match_id[i] == data.match_id[i-1] and data.inning[i] == data.inning[i-1]:
        innings_total = innings_total + data.total_runs[i]
        innings_total_list.append(innings_total)
        i = i+1

In [ ]:
data['cumulative_innings_total'] = innings_total_list 
data['cumulative_balls'] = data['over']*6 + data['ball']

In [ ]:
#get the total runs scored in that innings
data['innings_total_runs'] = data.groupby(['match_id', 'inning'])['cumulative_innings_total'].transform('max')

#get the total balls bowled in that innings
data['innings_total_balls'] = data.groupby(['match_id', 'inning'])['cumulative_balls'].transform('max')

In [ ]:
data['match_total_runs'] = data['target_runs'] - 1 + data['target_runs'] - 1 - data['result_margin']

In [ ]:
#get the first row in each innings and then add the total balls in each innings to get the total balls bowled in the match
data_first_rows = data.groupby(['match_id', 'inning']).first().reset_index()
data_first_rows['match_total_balls'] = data_first_rows.groupby('match_id')['innings_total_balls'].transform('sum')
to_merge_df = data_first_rows[['match_id', 'match_total_balls']].drop_duplicates()

In [ ]:
data = pd.merge(data, to_merge_df, on='match_id', how='left')

In [ ]:
data[0:10]

In [ ]:
data['batter_total_runs'] = data.groupby(['match_id','batter'])['batsman_runs'].cumsum()
data['batter_total_balls'] = data[(data['extras_type'] != 'wides') & (data['extras_type'] != 'penalty')].groupby(['match_id','batter']).cumcount() + 1
#will forward fill the last valid value to the nan value
data['batter_total_balls'] = data.groupby(['match_id', 'batter'])['batter_total_balls'].transform(lambda x: x.ffill())
#if the first ball that the batter faced was a wide then there is nothing to ffill so we fill these with 0
data['batter_total_balls'] = data['batter_total_balls'].fillna(0)

In [ ]:
data['bowler_total_runs'] = data[(data['extras_type'] != 'byes') & (data['extras_type'] != 'legbyes') & (data['extras_type'] != 'penalty')].groupby(['match_id', 'bowler'])['total_runs'].cumsum()
data['bowler_total_balls'] = data[(data['extras_type'] != 'wides') & (data['extras_type'] != 'noballs') & (data['extras_type'] != 'penalty')].groupby(['match_id','bowler']).cumcount() + 1
#will forward fill the last valid value to the nan value
data['bowler_total_runs'] = data.groupby(['match_id', 'bowler'])['bowler_total_runs'].transform(lambda x: x.ffill())
#if the first ball bowled by a bowler is a bye then there is nothing to ffill so we fill these with 0
data['bowler_total_runs'] = data['bowler_total_runs'].fillna(0)
#will forward fill the last valid value to the nan value
data['bowler_total_balls'] = data.groupby(['match_id', 'bowler'])['bowler_total_balls'].transform(lambda x: x.ffill())
#if the first ball bowled by a bowler is a noball then there is nothing to ffill so we fill these with 0
data['bowler_total_balls'] = data['bowler_total_balls'].fillna(0)

In [ ]:
data['bowler_economy'] = data['bowler_total_runs']/data['bowler_total_balls'] * 6

In [ ]:
data['batter_SR'] = data['batter_total_runs']/data['batter_total_balls'] * 100

In [ ]:
data.columns

In [ ]:
data.season.unique()

In [ ]:
data_2008 = data[data['season'] =='2007/08']
data_2009 = data[data['season'] =='2009']
data_2010 = data[data['season'] =='2009/10']
data_2011 = data[data['season'] =='2011']
data_2012 = data[data['season'] =='2012']
data_2013 = data[data['season'] =='2013']
data_2014 = data[data['season'] =='2014']
data_2015 = data[data['season'] =='2015']
data_2016 = data[data['season'] =='2016']
data_2017 = data[data['season'] =='2017']
data_2018 = data[data['season'] =='2018']
data_2019 = data[data['season'] =='2019']
data_2020 = data[data['season'] =='2020/21']
data_2021 = data[data['season'] =='2021']
data_2022 = data[data['season'] =='2022']
data_2023 = data[data['season'] =='2023']

In [ ]:
xR_data = data

In [ ]:
#xR_data.to_csv("xR_data.csv", index=False)

In [ ]:
unnecessary_cols = [
    'batting_team', 'batsman_runs', 'over', 'ball', 'total_runs',
    'non_striker', 'extra_runs', 'extras_type', 'is_wicket',
    'player_dismissed', 'fielder', 'season', 'city',
    'player_of_match', 'venue', 'team1', 'team2', 'toss_winner',
    'toss_decision', 'winner', 'result', 'result_margin', 'target_runs',
    'target_overs', 'super_over', 'method', 'umpire1', 'umpire2',
    'cumulative_innings_total', 'innings_total_runs',
    'innings_total_balls', 'match_total_runs', 'match_total_balls',
    'bowler_total_runs', 'bowler_total_balls', 'bowler_economy',
]

In [ ]:
# Drop the unnecessary columns
df_reduced = data.drop(columns=unnecessary_cols)

# Display the new DataFrame
df_reduced

In [ ]:
# ball_by_ball_data[0:10]
df_reduced['batter_score'] = df_reduced.groupby(['match_id', 'batter'])['batter_total_runs'].transform('max')
df_reduced

In [ ]:
dismissal_kinds = df_reduced['dismissal_kind'].unique()
dismissal_kinds

In [ ]:
import pandas as pd

# Read the player.csv file into a DataFrame
players = pd.read_csv("data/Player.csv")

# players

new_players = pd.DataFrame()
new_players[['Player_Name', 'Bowling_Skill']] = players[['Player_Name', 'Bowling_Skill']]

new_players.dropna(inplace=True)

# new_players

df_reduced['Bowling_Skill'] = df_reduced['bowler'].map(new_players.set_index('Player_Name')['Bowling_Skill'])
df_reduced['Bowling_Skill'] = df_reduced['Bowling_Skill'].fillna('nan')
df_reduced[20:40]

df_reduced['Bowling_Skill'] = df_reduced['Bowling_Skill'].replace({
  'Right-arm medium': 'Right-arm pace',
  'Right-arm fast-medium': 'Right-arm pace',
  'Right-arm medium-fast': 'Right-arm pace',
  'Right-arm fast': 'Right-arm pace',
  'Left-arm medium': 'Left-arm pace',
  'Left-arm fast-medium': 'Left-arm pace',
  'Left-arm medium-fast': 'Left-arm pace',
  'Left-arm fast': 'Left-arm pace',
  'Right-arm offbreak': 'Right-arm offspin',
  'Legbreak googly': 'Right-arm legspin',
  'Legbreak': 'Right-arm legspin',
  'Slow left-arm orthodox': 'Left-arm offspin',
  'Slow left-arm chinaman': 'Left-arm legspin',
})

unique_bowling_types = df_reduced['Bowling_Skill'].unique()
unique_bowling_types

In [ ]:
print(df_reduced.columns)

In [ ]:
# player name, runs scored in match, no. of each bowler type, for each batter: match-wise rows, batting pos, innings, venue

In [ ]:
df_reduced = df_reduced.loc[0:140]

In [ ]:
df_reduced['no_of_right_arm_pacers_faced'] = 0
df_reduced['no_of_left_arm_pacers_faced'] = 0
df_reduced['no_of_right_arm_offspinners_faced'] = 0
df_reduced['no_of_left_arm_offpacers_faced'] = 0
df_reduced['no_of_right_arm_legspinners_faced'] = 0
df_reduced['no_of_left_arm_legspinners_faced'] = 0


# Dictionary to keep track of bowlers faced by each batter in each match
bowlers_faced = {}

# Iterate through the DataFrame
for idx, row in df_reduced.iterrows():
    match_id = row['match_id']
    batter = row['batter']
    bowler = row['bowler']
    skill = row['Bowling_Skill']
    
    if match_id not in bowlers_faced:
        bowlers_faced[match_id] = {}
    
    if batter not in bowlers_faced[match_id]:
        bowlers_faced[match_id][batter] = {'Right-arm pace': set(), 'Left-arm pace': set(), 'Right-arm offspin': set(), 'Left-arm offspin': set(), 'Right-arm legspin': set(), 'Left-arm legspin': set(), 'nan': set()}
    
    #if bowler not in bowlers_faced[match_id][batter][skill]:
    bowlers_faced[match_id][batter][skill].add(bowler)
    
    df_reduced.loc[df_reduced.index == idx, 'no_of_right_arm_pacers_faced'] = len(bowlers_faced[match_id][batter]['Right-arm pace'])
    df_reduced.loc[df_reduced.index == idx, 'no_of_left_arm_pacers_faced'] = len(bowlers_faced[match_id][batter]['Left-arm pace'])
    df_reduced.loc[df_reduced.index == idx, 'no_of_right_arm_offspinners_faced'] = len(bowlers_faced[match_id][batter]['Right-arm offspin'])
    df_reduced.loc[df_reduced.index == idx, 'no_of_left_arm_offspinners_faced'] = len(bowlers_faced[match_id][batter]['Left-arm offspin'])
    df_reduced.loc[df_reduced.index == idx, 'no_of_right_arm_legspinners_faced'] = len(bowlers_faced[match_id][batter]['Right-arm legspin'])
    df_reduced.loc[df_reduced.index == idx, 'no_of_left_arm_legspinners_faced'] = len(bowlers_faced[match_id][batter]['Left-arm legspin'])
    # df_reduced.loc[df_reduced.index == idx, 'no_of_nans'] = len(bowlers_faced[match_id][batter]['nan'])

    # if skill == 'Right-arm pace':
    #     df_reduced.loc[df_reduced.index == idx, 'no_of_right_arm_pacers_faced'] = len(bowlers_faced[match_id][batter]['Right-arm pace'])
    # if skill == 'Left-arm pace':
    #     df_reduced.loc[df_reduced.index == idx, 'no_of_left_arm_pacers_faced'] = len(bowlers_faced[match_id][batter]['Left-arm pace'])
    # if skill == 'Right-arm offspin':
    #     df_reduced.loc[df_reduced.index == idx, 'no_of_right_arm_offspinners_faced'] = len(bowlers_faced[match_id][batter]['Right-arm offspin'])
    # if skill == 'Left-arm offspin':
    #     df_reduced.loc[df_reduced.index == idx, 'no_of_left_arm_offspinners_faced'] = len(bowlers_faced[match_id][batter]['Left-arm offspin'])
    # if skill == 'Right-arm legspin':
    #     df_reduced.loc[df_reduced.index == idx, 'no_of_right_arm_legspinners_faced'] = len(bowlers_faced[match_id][batter]['Right-arm legspin'])
    # if skill == 'Left-arm legspin':
    #     df_reduced.loc[df_reduced.index == idx, 'no_of_left_arm_legspinners_faced'] = len(bowlers_faced[match_id][batter]['Left-arm legspin'])
    # if skill == 'nan':
    #     df_reduced.loc[df_reduced.index == idx, 'no_of_nans'] = len(bowlers_faced[match_id][batter]['nan'])


In [ ]:
bowlers_faced

In [ ]:
df_reduced[120:140]